# JAST Demo

We will demonstrate the usage of the JAST library by parsing a simple Java source code, printing the AST, and modifying the AST.

We start by importing the JAST library.

In [2]:
import jast

Let's consider a simple Java class:
```java
class Add {
    public static void add(int a, int b) {
        System.out.println(a + b);
    }

    public static void main(String[] args) {
        System.out.println(add(27, 55));
    }
}
```
We can find an implementation of it in the `Add.java` file in the demo directory.

Let's get the content of the file.

In [3]:
import os
with open(os.path.join("demo", "Add.java")) as f:
    src = f.read()

In [4]:
print(src)

class Add {
    public static int add(int a, int b) {
        return a + b;
    }

    public static void main(String[] args) {
        System.out.println(add(27, 55));
    }
}


We can compile the Java file to execute it.

In [5]:
import subprocess
subprocess.run(["javac", "-d", "demo", os.path.join("demo", "Add.java")])
process = subprocess.run(["java", "-cp", "demo", "Add"], stdout=subprocess.PIPE)
print(process.stdout.decode())

82



Now we can parse the AST from the Java source code.

In [6]:
tree = jast.parse(src)

Let's take a look at the AST by un-parsing it.

In [7]:
print(jast.unparse(tree))

class Add {
    public static int add(int a, int b) {
        return a + b;
    }

    public static void main(String[] args) {
        System.out.println(add(27, 55));
    }
}



Next, we will modify the AST by making the `add` method return the difference of the two arguments instead of the sum.

In [10]:
class ChangeAdd(jast.JNodeTransformer):
    def visit_Method(self, node):
        if node.id == "add":
            node.body = self.visit(node.body)
            return node
        return node

    def visit_BinOp(self, node):
        if isinstance(node.op, jast.Add):
            node.op = jast.Sub()
            return node
        return node

In [11]:
change_add = ChangeAdd()

In [12]:
new_tree = change_add.visit(tree)

In [13]:
code = jast.unparse(new_tree)
print(code)

class Add {
    public static int add(int a, int b) {
        return a - b;
    }

    public static void main(String[] args) {
        System.out.println(add(27, 55));
    }
}



We can save the modified Java source code to a file, compile it, and execute it.

In [14]:
with open(os.path.join("demo", "Sub.java"), "w") as f:
    f.write(code)

In [15]:
subprocess.run(["javac", "-d", "demo", os.path.join("demo", "Sub.java")])

CompletedProcess(args=['javac', '-d', 'demo', 'demo/Sub.java'], returncode=0)

In [16]:
process = subprocess.run(["java", "-cp", "demo", "Add"], stdout=subprocess.PIPE)
print(process.stdout.decode())

-28

